# 선형 회귀식의 계수를 찾는 법 - OLS VS. SGD
- 보스턴 집값 데이터 활용(RM VS Price)

### 필요한 모듈 import

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 필요한 라이브러리 import 


In [ ]:
from sklearn import datasets

dir(datasets)

['__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__getattr__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_arff_parser',
 '_base',
 '_california_housing',
 '_covtype',
 '_kddcup99',
 '_lfw',
 '_olivetti_faces',
 '_openml',
 '_rcv1',
 '_samples_generator',
 '_species_distributions',
 '_svmlight_format_fast',
 '_svmlight_format_io',
 '_twenty_newsgroups',
 'clear_data_home',
 'dump_svmlight_file',
 'fetch_20newsgroups',
 'fetch_20newsgroups_vectorized',
 'fetch_california_housing',
 'fetch_covtype',
 'fetch_kddcup99',
 'fetch_lfw_pairs',
 'fetch_lfw_people',
 'fetch_olivetti_faces',
 'fetch_openml',
 'fetch_rcv1',
 'fetch_species_distributions',
 'get_data_home',
 'load_breast_cancer',
 'load_diabetes',
 'load_digits',
 'load_files',
 'load_iris',
 'load_linnerud',
 'load_sample_image',
 'load_sample_images',
 'load_svmlight_file',
 'load_svmlight_files',
 'load_wine',
 'make_biclusters',
 'make_blobs',
 'make_checkerboard',
 'make_circl

### 데이터 수집 및 분할

In [ ]:
from sklearn import datasets

boston = datasets.fetch_openml('boston')
boston.keys()

c:\Users\PlayData\anaconda3\lib\site-packages\sklearn\datasets\_openml.py:292: UserWarning: Multiple active versions of the dataset matching the name boston exist. Versions may be fundamentally different, returning version 1.
  warn(
c:\Users\PlayData\anaconda3\lib\site-packages\sklearn\datasets\_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [ ]:
X = boston.data
y = boston.target

# 데이터 분할

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [8]:
np.array(boston.data.RM).reshape(-1, 1)

array([[6.575],
       [6.421],
       [7.185],
       [6.998],
       [7.147],
       [6.43 ],
       [6.012],
       [6.172],
       [5.631],
       [6.004],
       [6.377],
       [6.009],
       [5.889],
       [5.949],
       [6.096],
       [5.834],
       [5.935],
       [5.99 ],
       [5.456],
       [5.727],
       [5.57 ],
       [5.965],
       [6.142],
       [5.813],
       [5.924],
       [5.599],
       [5.813],
       [6.047],
       [6.495],
       [6.674],
       [5.713],
       [6.072],
       [5.95 ],
       [5.701],
       [6.096],
       [5.933],
       [5.841],
       [5.85 ],
       [5.966],
       [6.595],
       [7.024],
       [6.77 ],
       [6.169],
       [6.211],
       [6.069],
       [5.682],
       [5.786],
       [6.03 ],
       [5.399],
       [5.602],
       [5.963],
       [6.115],
       [6.511],
       [5.998],
       [5.888],
       [7.249],
       [6.383],
       [6.816],
       [6.145],
       [5.927],
       [5.741],
       [5.966],
       [

In [9]:
X = np.array(boston.data.RM).reshape(-1, 1)
y = boston.target

# 데이터 분할

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# 1. LinearRegression 모델을 사용한 경우

In [10]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg.fit(X_train, y_train)
print(reg.coef_, reg.intercept_)

[8.46109164] -30.571032410898336


In [11]:
y_pred = reg.predict(X_test)

In [12]:
from evaluate_model import evaluate_score # 모듈을 만들어서 불러오다.
evaluate_score(y_test, y_pred)

mse: 36.517
rmse: 6.043
r2: 0.602


# 2. SGDRegressor with hyperparameter

In [101]:
from sklearn.linear_model import SGDRegressor

reg = SGDRegressor(max_iter=1000000000, eta0=0.001, learning_rate='invscaling', loss='squared_error', random_state=42)
reg.fit(X_train, y_train)

print(reg.coef_[0], reg.intercept_[0])

3.6558673360157155 -0.11398446076518967


In [102]:
y_pred = reg.predict(X_test)  # 한번 학습을 하고 나면 시작점이 달라지지 않는다.
evaluate_score(y_test, y_pred)

mse: 58.721
rmse: 7.663
r2: 0.359


# 3. SGDRegressor with scaling

In [105]:
# 표준화 스케일링을 사용하여 경사하강법 모델링

train_mean = np.mean(X_train, axis=0)
train_std = np.std(X_train, axis=0)

X_train_scaled = (X_train - train_mean) / train_std
X_test_scaled = (X_test - train_mean)/ train_std 

# 모델 객체 생성
reg = SGDRegressor(max_iter=1000000000, eta0=0.0001, learning_rate='invscaling', loss='squared_error', random_state=42)

reg.fit(X_train_scaled, y_train)

# 계수 확인
print(reg.coef_, reg.intercept_)

[5.62395551] [21.47086473]


In [106]:
# 평가 지표
y_pred = reg.predict(X_test_scaled)
evaluate_score(y_test, y_pred) # 엄청난 성능 증가가 발견된다...!

mse: 37.668
rmse: 6.137
r2: 0.589


# 4. SGDRegressor with StandardScaler()

In [107]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
# X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

array([[-4.40132662e-01],
       [-4.15554923e-01],
       [ 7.81525543e-01],
       [ 4.93821422e-01],
       [-3.89258212e+00],
       [ 4.11413708e-01],
       [-2.73871486e-01],
       [ 5.45868398e-01],
       [ 2.26486438e+00],
       [ 3.65278376e+00],
       [ 6.71648592e-01],
       [-3.57724948e-01],
       [-8.75303217e-01],
       [-3.38770929e-02],
       [ 3.79607223e-01],
       [ 1.82985310e-01],
       [-8.10244496e-01],
       [ 7.02168606e-02],
       [ 3.75269974e-01],
       [-6.80127054e-01],
       [-2.98449225e-01],
       [ 1.19356411e+00],
       [-7.81329509e-01],
       [-1.21908526e-02],
       [ 7.64176551e-01],
       [-5.39889367e-01],
       [ 1.78632134e+00],
       [-5.32660620e-01],
       [-1.29296550e-01],
       [ 2.19129044e-01],
       [ 5.83457882e-01],
       [ 1.23404509e+00],
       [-1.06035913e+00],
       [-3.86639935e-01],
       [-3.96600903e-02],
       [ 2.07836271e+00],
       [-4.71939147e-01],
       [-5.54346861e-01],
       [-5.2

# 5. Pipeline with StandardScaler, LinearRegression or SGDRegressor

In [108]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

reg = make_pipeline(StandardScaler(),
                    SGDRegressor(max_iter=1000000, eta0=0.01,\
                                tol=0.0001, random_state=42, loss='squared_error'))
reg.fit(X_train, y_train)

# 계수 및 절편 확인: _속성은 학습을 통해 결정되는 속성
print(reg[1].coef_, reg[1].intercept_)

#회귀식 - pipeline()을 사용했기 때문에 SGDRegressor의 parameter가 reg객체의 1번 인덱스에 들어감
print("y = {:2f}X + {:.3f}".format(reg[1].coef_[0], reg[1].intercept_[0]))

# 예측 수행
y_pred = reg.predict(X_test)
from sklearn.metrics import mean_squared_error, r2_score
# MSE, RMSE, r2_score
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("MSE:", np.round(mse, 3))
print("RMSE: ", np.round(rmse, 3))
print("R2: ", np.round(r2, 3))

[5.84750366] [22.31897879]
y = 5.847504X + 22.319
MSE: 36.523
RMSE:  6.043
R2:  0.602
